In [1]:
from torch import tensor, stft, hann_window, abs, square
from torch.cuda import is_available
from time import time
from os import makedirs

from utils_basic import SPECTROGRAM_DIR as outdir, HYDRO_STATIONS as stations
from utils_basic import get_geo_metadata, get_geophone_days
from utils_torch import get_hourly_hydro_spectrograms_for_a_day
from utils_spec import save_hydro_spectrograms
from utils_preproc import read_and_process_day_long_hydro_waveforms

In [2]:
# Inputs
cuda = False
window_length = 1.0
overlap = 0.5
downsample = False

In [3]:
# Check if CUDA is available
if cuda == True:
    if is_available():
        print("GPU is available")
    else:
        print("GPU is NOT available")

In [4]:
# Load the list of days
days = ["2020-01-13"]

In [5]:
# Create the output directory
makedirs(outdir, exist_ok = True)

In [6]:
# Loop over all days and stations
for day in days:
    for station in stations:
        print("######")
        print(f"Working on {day} for {station}...")
        print("######")
        
        clock1 = time()

        # Read and preprocess
        print(f"Reading and preprocessing the waveforms...")
        stream_day = read_and_process_day_long_hydro_waveforms(day, stations = station)

        if stream_day == None:
            print("No data found! Skipped.")
            continue

        # Compute the hourly spectrograms
        print(f"Computing hourly spectrograms...")
        stream_spec, stream_spec_ds = get_hourly_hydro_spectrograms_for_a_day(stream_day, 
                                                                            window_length = window_length, overlap = overlap, cuda = cuda, downsample = downsample)

        # Save the results     
        print("Saving the spectrograms...")
        starttimes = stream_spec.get_starttimes()
        for starttime in starttimes:
            stream_spec_hour = stream_spec.select(starttime = starttime)
            
            filename = f"hourly_hydro_spectrograms_{starttime}_{station}_window{window_length:.0f}s.h5"
            save_hydro_spectrograms(stream_spec_hour, filename)
        
        clock2 = time()
        
        elapse_time = clock2 - clock1
        print(f"Elapsed time: {elapse_time:f} s")

######
Working on 2020-01-13 for A00...
######
Reading and preprocessing the waveforms...
Reading the waveforms for 2020-01-13
Preprocessing the waveforms...
Computing hourly spectrograms...
Computing the spectrograms for starttime 2020-01-13T00:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T01:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T02:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T03:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T04:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T05:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T06:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T07:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T08:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T09:00:01.928000Z...
Computing the spectrograms for starttime 2020-01-13T10:00:01.928000Z...
Computing the spe

ValueError: Error: The spectrograms do not have the same start time!

In [24]:
starttimes = stream_spec.get_starttimes()
for starttime in starttimes:
    stream_spec_hour = stream_spec.select(starttime = starttime)
    for trace_spec_hour in stream_spec_hour:
        print(trace_spec_hour.times[0])
    print("")

2020-01-13 00:00:01.928000+00:00
2020-01-13 00:00:01.927999+00:00
2020-01-13 00:00:01.927999+00:00

2020-01-13 00:00:02.994000+00:00

2020-01-13 01:00:01.928000+00:00
2020-01-13 01:00:01.928000+00:00
2020-01-13 01:00:01.927999+00:00
2020-01-13 01:00:01.927999+00:00

2020-01-13 02:00:01.928000+00:00
2020-01-13 02:00:01.928000+00:00
2020-01-13 02:00:01.927999+00:00
2020-01-13 02:00:01.927999+00:00

2020-01-13 03:00:01.928000+00:00
2020-01-13 03:00:01.928000+00:00
2020-01-13 03:00:01.927999+00:00
2020-01-13 03:00:01.927999+00:00

2020-01-13 04:00:01.928000+00:00
2020-01-13 04:00:01.928000+00:00
2020-01-13 04:00:01.927999+00:00
2020-01-13 04:00:01.927999+00:00

2020-01-13 05:00:01.928000+00:00
2020-01-13 05:00:01.928000+00:00
2020-01-13 05:00:01.927999+00:00
2020-01-13 05:00:01.927999+00:00

2020-01-13 06:00:01.928000+00:00
2020-01-13 06:00:01.928000+00:00
2020-01-13 06:00:01.927999+00:00
2020-01-13 06:00:01.927999+00:00

2020-01-13 07:00:01.928000+00:00
2020-01-13 07:00:01.928000+00:00
20